In [1]:
import numpy as np
import random

# Generate reward table for reinforcement learning

In [2]:
def generate_reward_map(game_map):
    reward_map = game_map.copy()
    
    reward_map[reward_map=='blank'] = -0.1
    reward_map[reward_map=='rock'] = -5
    reward_map[reward_map=='monster'] = -10
    reward_map[reward_map=='treasure'] = 10
    
    reward_map = reward_map.astype(float)
    
    return reward_map

The update_next_state function will update and check for next state, if the next state is out of range, (eg. current position at (0,0), but action is UP, or LEFT), then stay at current state.

In [3]:
def update_next_state(current_state, action,maxedge):
    next_state = current_state
    if action == 0 :
        next_state = (current_state[0] - 1, current_state[1])
    elif action == 1 :
        next_state = (current_state[0] + 1, current_state[1])
    elif action == 2 :
        next_state = (current_state[0], current_state[1] - 1)
    elif action == 3 :
        next_state = (current_state[0], current_state[1] + 1)
    
    # check if next state is out of range
    if next_state[0] < 0 or next_state[1] < 0 or next_state[0] > maxedge[0]-1 or next_state[1] > maxedge[1]-1:
        next_state = current_state

    return next_state

# Now start Q_learning module for reinforcement learning

$Q(S_t, A_t) = Q(S_t, A_t) + \alpha [ \, R_{t+1} + \gamma \max_{a} Q(S_{t+1}, a) - Q(S_t, A_t)] \,$

In [4]:
def Q_learning(alpha, gamma, start_state, end_state, reward_map, epochs):
    n, p = reward_map.shape
    
    # initialize Q_table
    Q_value = [0,0,0,0]
    Q_table = np.zeros((n,p,4))

    for i in range(epochs):
        current_state = start_state
        while current_state != end_state:
            # randomly pick an action
            action = random.randint(0,3)
            # Q learning
            next_state = update_next_state(current_state,action,(n,p))
            Q_next = Q_table[next_state[0], next_state[1]]
            reward_after_action = reward_map[next_state[0], next_state[1]]

            old_Q = Q_table[current_state[0], current_state[1]][action]

            new_Q = old_Q + alpha*(reward_after_action + gamma * max(Q_next) - old_Q)
            Q_table[current_state[0], current_state[1]][action] = new_Q

            current_state = next_state
    return Q_table

### Based on the Q_table after learning, find the optimal path that avoid obstacles and opponents with less step

In [5]:
def get_best_path(Q_table,start,end):
    n, p, z= Q_table.shape
    current = start
    best_path = []
    while current != end:
        Q_value_current = Q_table[current[0],current[1]]
        action = np.argmax(Q_value_current)
        best_path.append(action)
        next_state = update_next_state(current,action,(n,p))
        current = next_state
    return best_path

# Implement reinforcement learning

In [6]:
def rl_algorithm(game_map, start_point, alpha, gamma, epochs):

    index = np.where(game_map == 'treasure')
    index_tu = list(zip(index[0], index[1]))
    end_point = index_tu[0]
    
    reward_map = generate_reward_map(game_map)
    Q_table = Q_learning(alpha, gamma, start_point, end_point, reward_map, epochs)
    best_path = get_best_path(Q_table,start_point, end_point)
    
    print("Done reinforcement learning!")
    print("Printing Q_table: ")
    print(Q_table)
    print(' ')
    print(['The optimal path to treature from ', start_point, ' is: ', best_path])
    print(" ")
    return Q_table, best_path

### print out the original map with out optimal path

"-" means blank cell  
"#" is tree   
"X" is monsters   
"$" is treasure   

In [7]:
def print_graph(game_map):
    graph = game_map.copy()
    graph[graph=='blank'] = '-'
    graph[graph=='rock'] = '#'
    graph[graph=='monster'] = 'X'
    graph[graph=='treasure'] = '$'
    
    print("Printing the map without path:")
    print(graph)
    print(" ")
    return graph

### Print out the map with optimal path

In [8]:
def print_graph_with_path(graph,best_path,current):
    for action in best_path:
        if action == 0:
            graph[current[0],current[1]] = "^"
        elif action == 1:
            graph[current[0],current[1]] = "v"
        elif action == 2:
            graph[current[0],current[1]] = "<"
        elif action == 3:
            graph[current[0],current[1]] = ">"

        current = update_next_state(current,action,graph.shape)
        
    print("Printing the map with optimal path:")
    print(graph)
    print(" ")

In [9]:
blank = 'blank'
rock = 'rock'
monster = 'monster'
treasure = 'treasure'
start = (5,0)

game_map = np.array([[blank, rock, treasure, blank, blank, blank],
           [blank, rock, blank, rock, blank, blank],
           [blank, blank, blank, blank, rock, blank],
           [rock, blank, monster, blank, blank, blank],
           [monster, blank, blank, rock, blank, rock],
           [blank, blank, rock, blank, blank, blank]])

graph = print_graph(game_map)

Q_table, best_path = rl_algorithm(game_map,start,0.5,0.8,100)

print_graph_with_path(graph,best_path,start)

Printing the map without path:
[['-' '#' '$' '-' '-' '-']
 ['-' '#' '-' '#' '-' '-']
 ['-' '-' '-' '-' '#' '-']
 ['#' '-' 'X' '-' '-' '-']
 ['X' '-' '-' '#' '-' '#']
 ['-' '-' '#' '-' '-' '-']]
 
Done reinforcement learning!
Printing Q_table: 
[[[ 2.29999999  2.94064     2.3         3.        ]
  [ 3.          1.32        2.3        10.        ]
  [ 0.          0.          0.          0.        ]
  [ 7.89998936  1.31999904  9.9999997   6.21994795]
  [ 6.21998972  4.87598556  7.89999885  4.87599211]
  [ 4.87599264  3.80079307  6.21999627  4.87599328]]

 [[ 2.29999999  3.8008      2.94064     1.32      ]
  [ 3.          4.876       2.94064     7.9       ]
  [10.          6.22        1.32        1.31999968]
  [ 7.89999972  4.876       7.89999998  4.87599591]
  [ 6.21999653 -1.0992      1.31999994  3.80079319]
  [ 4.87599129  2.94063484  4.87599518  3.80079526]]

 [[ 2.94064    -1.95936     3.8008      4.876     ]
  [ 1.32        3.8008      3.8008      6.22      ]
  [ 7.9        -5.024   

#### Now find the best path from different entry point

In [10]:
start = (0,0)
graph = print_graph(game_map)

Q_table, best_path = rl_algorithm(game_map,start,0.5,0.8,100)

print_graph_with_path(graph,best_path,start)

Printing the map without path:
[['-' '#' '$' '-' '-' '-']
 ['-' '#' '-' '#' '-' '-']
 ['-' '-' '-' '-' '#' '-']
 ['#' '-' 'X' '-' '-' '-']
 ['X' '-' '-' '#' '-' '#']
 ['-' '-' '#' '-' '-' '-']]
 
Done reinforcement learning!
Printing Q_table: 
[[[ 2.3         2.94063871  2.3         3.        ]
  [ 3.          1.31999998  2.3        10.        ]
  [ 0.          0.          0.          0.        ]
  [ 7.88652344  1.31962101  9.99984741  6.08931519]
  [ 6.16166534  4.86124613  7.89563637  4.84566641]
  [ 4.85630265  3.78997946  6.20774376  4.79169927]]

 [[ 2.3         3.80079924  2.94063846  1.32      ]
  [ 3.          4.87599902  2.9406339   7.9       ]
  [10.          6.2199971   1.31999973  1.31997123]
  [ 7.89957766  4.87599668  7.89999882  4.86922233]
  [ 6.21286562 -1.09921645  1.31999383  3.79273571]
  [ 4.85928539  2.93212394  4.87013897  3.79427441]]

 [[ 2.94063829 -1.95936153  3.80079974  4.87599983]
  [ 1.31999999  3.80079848  3.80079944  6.21999986]
  [ 7.8999999  -5.024000

#### Try with different map

In [11]:
start = (0,0)
game_map = np.array([[blank, rock, rock, blank, blank],
                    [blank, rock, blank, blank, blank],
                    [blank, blank, blank, rock, blank],
                    [blank, rock, blank, rock, blank],
                    [blank, monster, blank, blank, treasure]])

graph = print_graph(game_map)

Q_table, best_path = rl_algorithm(game_map,start,0.5,0.8,100)

print_graph_with_path(graph,best_path,start)

Printing the map without path:
[['-' '#' '#' '-' '-']
 ['-' '#' '-' '-' '-']
 ['-' '-' '-' '#' '-']
 ['-' '#' '-' '#' '-']
 ['-' 'X' '-' '-' '$']]
 
Done reinforcement learning!
Printing Q_table: 
[[[ 1.26160768  1.7020096   1.26160768 -3.99071386]
  [-3.99071386 -2.647488    1.26160768 -2.647488  ]
  [-2.647488    2.94064    -3.99071386  2.94064   ]
  [ 2.94064     3.8008     -2.647488    3.8008    ]
  [ 3.8008      4.876       2.94064     3.8008    ]]

 [[ 1.26160768  2.252512    1.7020096  -2.647488  ]
  [-3.99071386  2.94064     1.7020096   2.94064   ]
  [-2.647488    3.8008     -2.647488    3.8008    ]
  [ 2.94064    -0.024       2.94064     4.876     ]
  [ 3.8008      6.22        3.8008      4.876     ]]

 [[ 1.7020096   1.7020096   2.252512    2.94064   ]
  [-2.647488   -1.0992      2.252512    3.8008    ]
  [ 2.94064     4.876       2.94064    -0.024     ]
  [ 3.8008      1.32        3.8008      6.22      ]
  [ 4.876       7.9        -0.024       6.22      ]]

 [[ 2.252512    1